<a href="https://colab.research.google.com/github/ijaycyndy/Data-Cleaning-Using-SQL/blob/main/Pozitive_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:

import pandas as pd
import numpy as np


import pandas as pd

# 1  read payment_month as plain text
df = pd.read_csv(
        "/content/sample_data/collab payment data.csv",
        dtype={"payment_month": "string"}
)

# 2  parse contract_end_date (DD/MM/YYYY)
df["contract_end_date"] = pd.to_datetime(
        df["contract_end_date"],
        dayfirst=True,
        errors="raise"
)

# 3  custom parser for the mixed payment_month strings
def parse_payment(s: str):
    s = s.strip()
    if "-" in s:
        # pattern like 2023-01-05  (Y-D-M)
        return pd.to_datetime(s, format="%Y-%d-%m", errors="coerce")
    elif "/" in s:
        # pattern like 01/05/2023  (D/M/Y)
        return pd.to_datetime(s, dayfirst=True, errors="coerce")
    else:
        return pd.NaT          # unexpected layout → NaT

df["payment_month"] = df["payment_month"].apply(parse_payment)

# ----- sanity check: raise if any payment_month failed to parse -----
if df["payment_month"].isna().any():
    bad_rows = df[df["payment_month"].isna()].head()
    raise ValueError(
        "Some payment_month strings could not be parsed. "
        f"Sample bad rows:\n{bad_rows}"
    )
# -------------------------------------------------------------------


# 4  retain raw text for audit
df["payment_month_raw"] = df["payment_month"].dt.strftime("%Y-%m-%d")

# 5  remove zeros & flag clawbacks
df = df[df["amount_paid"] != 0]
df["is_clawback"] = df["amount_paid"] < 0

df["contract_status"] = np.where(
    df["contract_end_date"] < pd.Timestamp("2025-07-01"),
    "inactive",
    "active"
)

df

/tmp/ipython-input-58-823520061.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_clawback"] = df["amount_paid"] < 0
/tmp/ipython-input-58-823520061.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["contract_status"] = np.where(


,supplier,mpan_mprn,contract_end_date,amount_paid,payment_month,Payment_Type,payment_month_raw,is_clawback,contract_status
0,Pozitive Energy,9341669006,2023-07-10,0.75,2023-05-01,Recurring,2023-05-01,False,inactive
1,Pozitive Energy,4232717110,2023-08-08,0.31,2024-05-01,Recurring,2024-05-01,False,inactive
3,Pozitive Energy,1824156410,2024-07-31,0.79,2023-08-01,Recurring,2023-08-01,False,inactive
5,Pozitive Energy,9341669006,2023-07-10,0.20,2022-11-01,Recurring,2022-11-01,False,inactive
7,Pozitive Energy,2520115710,2026-01-31,0.50,2025-04-01,Recurring,2025-04-01,False,active
...,...,...,...,...,...,...,...,...,...
23882,TotalEnergies,1100010974209,2026-03-31,579.67,2025-02-28,Recurring,2025-02-28,False,active
23883,TotalEnergies,1507954199998,2024-09-30,967.09,2024-09-30,Recurring,2024-09-30,False,inactive
23884,TotalEnergies,1416632240000,2025-12-31,86.86,2025-01-31,Recurring,2025-01-31,False,active
23885,TotalEnergies,2700001843909,2026-07-31,735.51,2025-02-28,Recurring,2025-02-28,False,active


In [ ]:
check = df[df["mpan_mprn"] == 1848605]

check

,supplier,mpan_mprn,contract_end_date,amount_paid,payment_month,payment_month_raw,is_clawback
1005,Pozitive Energy,1848605,2024-02-05,25.58,2024-01-01,2024-01-01,False
2439,Pozitive Energy,1848605,2024-02-05,30.13,2024-02-01,2024-02-01,False
2450,Pozitive Energy,1848605,2024-02-05,24.07,2024-02-01,2024-02-01,False
4956,Pozitive Energy,1848605,2024-02-05,88.14,2023-04-01,2023-04-01,False
5530,Pozitive Energy,1848605,2024-02-05,18.01,2024-04-01,2024-04-01,False
6423,Pozitive Energy,1848605,2024-02-05,2.52,2023-05-01,2023-05-01,False
8163,Pozitive Energy,1848605,2024-02-05,23.06,2023-06-01,2023-06-01,False
10086,Pozitive Energy,1848605,2024-02-05,15.32,2023-07-01,2023-07-01,False
11692,Pozitive Energy,1848605,2024-02-05,8.42,2023-08-01,2023-08-01,False
14209,Pozitive Energy,1848605,2024-02-05,11.11,2023-09-01,2023-09-01,False


column	meaning
gross_amount	sum of positive payments in that month
clawback_amount	sum of negative payments (refunds)
net_amount	gross + clawback (true cash effect)
num_payments	payment count

In [59]:
# 1. Create month key
df["month_period"] = df["payment_month"].dt.to_period("M")

# 2. Aggregate per MPAN-month-supplier
monthly = (
    df.groupby(["supplier", "mpan_mprn", "month_period"])
      .agg(
          gross_amount    = ("amount_paid",   lambda s: s[s > 0].sum()),
          net_amount      = ("amount_paid",   "sum"),
          clawback_amount = ("amount_paid",   lambda s: s[s < 0].sum()),
          num_payments    = ("amount_paid",   "size")
      )
      .reset_index()
)

# 3. Build calendar per mpan-supplier up to June 2025
last_period = pd.Period("2025-06", freq="M")

rows = [
    pd.DataFrame({
        "mpan_mprn": g["mpan_mprn"].iloc[0],
        "supplier": g["supplier"].iloc[0],
        "month_period": pd.period_range(g["month_period"].min(), last_period, freq="M")
    })
    for _, g in monthly.groupby(["mpan_mprn", "supplier"])
]

calendar = pd.concat(rows, ignore_index=True)

# 4. Merge supplier info and contract details into calendar
calendar = calendar.merge(
    df[["mpan_mprn", "supplier", "contract_end_date", "Payment_Type", "contract_status"]]
      .drop_duplicates(),
    on=["mpan_mprn", "supplier"],
    how="left"
)

# 5. Merge monthly aggregation and fill blanks
payment_aggregation = (
    calendar.merge(monthly, on=["mpan_mprn", "supplier", "month_period"], how="left")
            .fillna({
                "gross_amount":    0,
                "net_amount":      0,
                "clawback_amount": 0,
                "num_payments":    0
            })
)

payment_aggregation


,mpan_mprn,supplier,month_period,contract_end_date,Payment_Type,contract_status,gross_amount,net_amount,clawback_amount,num_payments
0,1848605,EDF,2024-05,2025-02-05,Upfront,inactive,208.61,208.61,0.0,1.0
1,1848605,EDF,2024-06,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
2,1848605,EDF,2024-07,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
3,1848605,EDF,2024-08,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
4,1848605,EDF,2024-09,2025-02-05,Upfront,inactive,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
30220,2700008316565,Pozitive Energy,2025-02,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30221,2700008316565,Pozitive Energy,2025-03,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30222,2700008316565,Pozitive Energy,2025-04,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0
30223,2700008316565,Pozitive Energy,2025-05,2024-04-30,Recurring,inactive,0.00,0.00,0.0,0.0


In [ ]:
check2 = train[train["mpan_mprn"] == 5297805]

check2

,mpan_mprn,month_period,gross_amount,net_amount,clawback_amount,num_payments
45,5297805,2023-10,31.48,31.48,0.00,2.0
46,5297805,2023-11,29.96,29.96,0.00,1.0
47,5297805,2023-12,72.04,72.04,0.00,1.0
48,5297805,2024-01,138.69,138.69,0.00,1.0
49,5297805,2024-02,163.77,163.77,0.00,1.0
50,5297805,2024-03,179.59,179.59,0.00,1.0
51,5297805,2024-04,144.75,144.75,0.00,1.0
52,5297805,2024-05,138.35,138.35,0.00,1.0
53,5297805,2024-06,100.31,100.31,0.00,1.0
54,5297805,2024-07,67.42,67.42,0.00,1.0


In [60]:
# Make a copy
training_data = payment_aggregation.copy()

# Sort for rolling logic
training_data = training_data.sort_values(["mpan_mprn", "supplier", "month_period"])

# 1. Month + Season
training_data["month_number"] = training_data["month_period"].dt.month
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}
training_data["season"] = training_data["month_number"].map(season_map)

# 2. Rolling Features — now grouped by both mpan and supplier
training_data["roll_3m_net"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["net_amount"]
                 .rolling(window=3, min_periods=1)
                 .sum()
                 .reset_index(level=[0, 1], drop=True)
)

training_data["roll_3m_avg"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["net_amount"]
                 .rolling(window=3, min_periods=1)
                 .mean()
                 .reset_index(level=[0, 1], drop=True)
)

training_data["roll_3m_gross"] = (
    training_data.groupby(["mpan_mprn", "supplier"])["gross_amount"]
                 .rolling(window=3, min_periods=1)
                 .sum()
                 .reset_index(level=[0, 1], drop=True)
)

# 3. Months since previous payment — grouped by both mpan and supplier
period_code = training_data["month_period"].astype(int)
training_data["months_since_prev_payment"] = (
    period_code.groupby([training_data["mpan_mprn"], training_data["supplier"]]).diff()
)

# 4. Reorder
cols_to_keep = [
    "mpan_mprn", "supplier", "month_period",
    "net_amount", "gross_amount", "clawback_amount", "num_payments",
    "contract_end_date", "contract_status", "Payment_Type",  # ← fix: add comma here
    "month_number", "season",
    "roll_3m_net", "roll_3m_avg", "roll_3m_gross",
    "months_since_prev_payment"
]

training_data = training_data[cols_to_keep]

training_data = training_data[
    (training_data["contract_status"] == "active") |
    (training_data["month_period"].dt.to_timestamp() <= training_data["contract_end_date"])
]

training_data

,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment
0,1848605,EDF,2024-05,208.61,208.61,0.00,1.0,2025-02-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN
1,1848605,EDF,2024-06,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,6,Summer,208.61,104.305000,208.61,1.0
2,1848605,EDF,2024-07,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,7,Summer,208.61,69.536667,208.61,1.0
3,1848605,EDF,2024-08,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,8,Summer,0.00,0.000000,0.00,1.0
4,1848605,EDF,2024-09,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30206,2700008316565,Pozitive Energy,2023-12,13.83,13.83,0.00,1.0,2024-04-30,inactive,Recurring,12,Winter,13.85,4.616667,29.03,1.0
30207,2700008316565,Pozitive Energy,2024-01,10.68,10.68,0.00,1.0,2024-04-30,inactive,Recurring,1,Winter,24.51,8.170000,24.51,1.0
30208,2700008316565,Pozitive Energy,2024-02,-24.47,0.01,-24.48,2.0,2024-04-30,inactive,Recurring,2,Winter,0.04,0.013333,24.52,1.0
30209,2700008316565,Pozitive Energy,2024-03,2.96,2.96,0.00,1.0,2024-04-30,inactive,Recurring,3,Spring,-10.83,-3.610000,13.65,1.0


In [61]:
training_data = training_data.sort_values(["mpan_mprn", "supplier", "month_period"])


training_data["next_month_payment"] = (
    training_data
    .groupby(["mpan_mprn", "supplier"])["net_amount"]
    .shift(-1)
)


training_data["will_pay_next_month"] = (
    training_data["next_month_payment"] > 0
).astype(int)


training_data


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month
0,1848605,EDF,2024-05,208.61,208.61,0.00,1.0,2025-02-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN,0.00,0
1,1848605,EDF,2024-06,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,6,Summer,208.61,104.305000,208.61,1.0,0.00,0
2,1848605,EDF,2024-07,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,7,Summer,208.61,69.536667,208.61,1.0,0.00,0
3,1848605,EDF,2024-08,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,8,Summer,0.00,0.000000,0.00,1.0,0.00,0
4,1848605,EDF,2024-09,0.00,0.00,0.00,0.0,2025-02-05,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,1.0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30206,2700008316565,Pozitive Energy,2023-12,13.83,13.83,0.00,1.0,2024-04-30,inactive,Recurring,12,Winter,13.85,4.616667,29.03,1.0,10.68,1
30207,2700008316565,Pozitive Energy,2024-01,10.68,10.68,0.00,1.0,2024-04-30,inactive,Recurring,1,Winter,24.51,8.170000,24.51,1.0,-24.47,0
30208,2700008316565,Pozitive Energy,2024-02,-24.47,0.01,-24.48,2.0,2024-04-30,inactive,Recurring,2,Winter,0.04,0.013333,24.52,1.0,2.96,1
30209,2700008316565,Pozitive Energy,2024-03,2.96,2.96,0.00,1.0,2024-04-30,inactive,Recurring,3,Spring,-10.83,-3.610000,13.65,1.0,4.39,1


In [79]:
check3 = training_data[
    (training_data["mpan_mprn"] == 2700001843909) &
    (training_data["supplier"] == "TotalEnergies")
]

check3


,mpan_mprn,supplier,month_period,net_amount,gross_amount,clawback_amount,num_payments,contract_end_date,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment,next_month_payment,will_pay_next_month
30149,2700001843909,TotalEnergies,2024-07,908.25,908.25,0.0,1.0,2024-07-31,inactive,Recurring,7,Summer,908.25,908.25,908.25,NaN,908.25,1
30150,2700001843909,TotalEnergies,2024-07,908.25,908.25,0.0,1.0,2026-07-31,active,Recurring,7,Summer,1816.50,908.25,1816.50,0.0,0.00,0
30152,2700001843909,TotalEnergies,2024-08,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,8,Summer,908.25,302.75,908.25,0.0,0.00,0
30154,2700001843909,TotalEnergies,2024-09,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,9,Autumn,0.00,0.00,0.00,0.0,0.00,0
30156,2700001843909,TotalEnergies,2024-10,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,10,Autumn,0.00,0.00,0.00,0.0,0.00,0
30158,2700001843909,TotalEnergies,2024-11,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,11,Autumn,0.00,0.00,0.00,0.0,0.00,0
30160,2700001843909,TotalEnergies,2024-12,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,12,Winter,0.00,0.00,0.00,0.0,0.00,0
30162,2700001843909,TotalEnergies,2025-01,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,1,Winter,0.00,0.00,0.00,0.0,735.51,1
30164,2700001843909,TotalEnergies,2025-02,735.51,735.51,0.0,1.0,2026-07-31,active,Recurring,2,Winter,1471.02,490.34,1471.02,0.0,0.00,0
30166,2700001843909,TotalEnergies,2025-03,0.00,0.00,0.0,0.0,2026-07-31,active,Recurring,3,Spring,735.51,245.17,735.51,0.0,0.00,0





MODEL TRAINING(for selecting the best hyper parameters)

In [29]:

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# --- Classifier ---
clf = RandomForestClassifier(random_state=42)

param_dist_clf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_clf = RandomizedSearchCV(
    clf,
    param_distributions=param_dist_clf,
    n_iter=10,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

random_search_clf.fit(Xc_train, yc_train)

# Best classifier model
best_clf = random_search_clf.best_estimator_
y_pred_class = best_clf.predict(Xc_test)
accuracy = accuracy_score(yc_test, y_pred_class)
print("Best Classifier Accuracy:", round(accuracy * 100, 2), "%")
print("Best Classifier Params:", random_search_clf.best_params_)


# --- Regressor ---
reg = RandomForestRegressor(random_state=42)

param_dist_reg = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search_reg = RandomizedSearchCV(
    reg,
    param_distributions=param_dist_reg,
    n_iter=10,
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search_reg.fit(Xr_train, yr_train)

# Best regressor model
best_reg = random_search_reg.best_estimator_
y_pred_reg = best_reg.predict(Xr_test)
rmse = np.sqrt(mean_squared_error(yr_test, y_pred_reg))
print("Best Regressor RMSE:", round(rmse, 2))
print("Best Regressor Params:", random_search_reg.best_params_)



KeyboardInterrupt: 

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

model_data = training_data.copy()

# --- Encode binary features ---
model_data["is_upfront"] = (model_data["Payment_Type"].str.lower() == "upfront").astype(int)
model_data["contract_status_bin"] = (model_data["contract_status"].str.lower() == "active").astype(int)

# --- One-hot encode season and supplier ---
season_dummies = pd.get_dummies(model_data["season"], prefix="season")
supplier_dummies = pd.get_dummies(model_data["supplier"], prefix="supplier")
model_data = pd.concat([model_data, season_dummies, supplier_dummies], axis=1)

# --- Define feature columns ---
features = [
    "month_number",
    "roll_3m_net",
    "roll_3m_avg",
    "roll_3m_gross",
    "months_since_prev_payment",
    "contract_status_bin",
    "is_upfront"
] + list(season_dummies.columns) + list(supplier_dummies.columns)

# --- Drop missing target values ---
model_data = model_data.dropna(subset=["will_pay_next_month", "next_month_payment"])

# --- Classification Data ---
X_class = model_data[features]
y_class = model_data["will_pay_next_month"]
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# --- Regression Data (only for rows where payment is expected) ---
X_reg = model_data[model_data["will_pay_next_month"] == 1][features]
y_reg = model_data[model_data["will_pay_next_month"] == 1]["next_month_payment"]
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# ======================
# TRAIN CLASSIFIER (using best hyperparameters)
# ======================
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=4,
    random_state=42
)
clf.fit(Xc_train, yc_train)
y_pred_class = clf.predict(Xc_test)
accuracy = accuracy_score(yc_test, y_pred_class)
print("Classification Accuracy:", round(accuracy * 100, 2), "%")

# ======================
# TRAIN REGRESSOR (using best hyperparameters)
# ======================
reg = RandomForestRegressor(
    n_estimators=200,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=2,
    random_state=42
)
reg.fit(Xr_train, yr_train)
y_pred_reg = reg.predict(Xr_test)
rmse = np.sqrt(mean_squared_error(yr_test, y_pred_reg))
print("Regression RMSE:", round(rmse, 2))


Classification Accuracy: 85.75 %
Regression RMSE: 128.34


Forecasting Script : preparing the date for forecasting

In [80]:
import pandas as pd

# Start with a copy
base_data = training_data.copy()

# STEP 1: Clean and standardize month_period
if isinstance(base_data["month_period"].iloc[0], pd.Period):
    base_data["month_period"] = base_data["month_period"].dt.to_timestamp()
else:
    base_data["month_period"] = pd.to_datetime(base_data["month_period"], errors="coerce")

# Drop rows with invalid month_period
base_data = base_data.dropna(subset=["month_period"])

# STEP 2: Filter to only active contracts
active_contracts = base_data[base_data["contract_status"].str.lower() == "active"]

# STEP 3: Get existing (mpan, month) combos to avoid duplicates
existing_mpan_months = set(zip(
    base_data["mpan_mprn"],
    base_data["month_period"].dt.to_period("M")
))

# STEP 4: Create new rows up to contract_end_date
future_rows = []

for _, row in active_contracts.iterrows():
    mpan = row["mpan_mprn"]
    start = row["month_period"] + pd.DateOffset(months=1)
    end = pd.to_datetime(row["contract_end_date"]).replace(day=1)
    forecast_months = pd.date_range(start=start, end=end, freq="MS")

    for date in forecast_months:
        period = date.to_period("M")
        if (mpan, period) in existing_mpan_months:
            continue  # Skip existing month

        new_row = {
            "mpan_mprn": mpan,
            "supplier": row["supplier"],
            "month_period": date,
            "month_number": date.month,
            "season": (
                "winter" if date.month in [12, 1, 2] else
                "spring" if date.month in [3, 4, 5] else
                "summer" if date.month in [6, 7, 8] else
                "autumn"
            ),
            "roll_3m_net": 0.0,
            "roll_3m_avg": 0.0,
            "roll_3m_gross": 0.0,
            "months_since_prev_payment": 1.0,
            "contract_status": row["contract_status"],
            "contract_end_date": row["contract_end_date"],
            "Payment_Type": row["Payment_Type"]
        }
        future_rows.append(new_row)

# STEP 5: Combine base and extended data
forecast_extension_df = pd.DataFrame(future_rows)
forecast_data = pd.concat([base_data, forecast_extension_df], ignore_index=True)

# STEP 6: Format month_period as YYYY-MM string
forecast_data["month_period"] = pd.to_datetime(forecast_data["month_period"], errors="coerce")
forecast_data = forecast_data.dropna(subset=["month_period"])
forecast_data["month_period"] = forecast_data["month_period"].dt.to_period("M").astype(str)

# STEP 7: Drop unused columns before prediction
forecast_data = forecast_data.drop(columns=[
    "net_amount",
    "gross_amount",
    "clawback_amount",
    "num_payments",
    "next_month_payment",
    "will_pay_next_month",
    "contract_end_date"
])

forecast_data.reset_index(drop=True, inplace=True)
forecast_data = forecast_data.drop_duplicates(subset=["mpan_mprn", "month_period"])


# Final result
forecast_data


,mpan_mprn,supplier,month_period,contract_status,Payment_Type,month_number,season,roll_3m_net,roll_3m_avg,roll_3m_gross,months_since_prev_payment
0,1848605,EDF,2024-05,inactive,Upfront,5,Spring,208.61,208.610000,208.61,NaN
1,1848605,EDF,2024-06,inactive,Upfront,6,Summer,208.61,104.305000,208.61,1.0
2,1848605,EDF,2024-07,inactive,Upfront,7,Summer,208.61,69.536667,208.61,1.0
3,1848605,EDF,2024-08,inactive,Upfront,8,Summer,0.00,0.000000,0.00,1.0
4,1848605,EDF,2024-09,inactive,Upfront,9,Autumn,0.00,0.000000,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
196181,2700001843909,TotalEnergies,2026-03,active,Recurring,3,spring,0.00,0.000000,0.00,1.0
196182,2700001843909,TotalEnergies,2026-04,active,Recurring,4,spring,0.00,0.000000,0.00,1.0
196183,2700001843909,TotalEnergies,2026-05,active,Recurring,5,spring,0.00,0.000000,0.00,1.0
196184,2700001843909,TotalEnergies,2026-06,active,Recurring,6,summer,0.00,0.000000,0.00,1.0


In [83]:
# ----------------------------
# FORECASTING SCRIPT

future_data = forecast_data.copy()

# 1. Feature engineering (same steps used in training)
future_data["is_upfront"] = (future_data["Payment_Type"].str.lower() == "upfront").astype(int)
future_data["contract_status_bin"] = (future_data["contract_status"].str.lower() == "active").astype(int)

# 2. One-hot encode season and supplier
season_dummies_future = pd.get_dummies(future_data["season"], prefix="season")
supplier_dummies_future = pd.get_dummies(future_data["supplier"], prefix="supplier")

# 3. Align columns with training one-hot structure
for col in season_dummies.columns:
    if col not in season_dummies_future:
        season_dummies_future[col] = 0
season_dummies_future = season_dummies_future[season_dummies.columns]

for col in supplier_dummies.columns:
    if col not in supplier_dummies_future:
        supplier_dummies_future[col] = 0
supplier_dummies_future = supplier_dummies_future[supplier_dummies.columns]

# 4. Concatenate all features
future_data = pd.concat([future_data, season_dummies_future, supplier_dummies_future], axis=1)

# 5. Final feature matrix for prediction
X_future = future_data[features]

# 6. Predict classification (whether supplier will pay)
future_data["will_pay_next_month"] = clf.predict(X_future)

# 7. Predict payment amount only for rows with expected payment
X_reg_future = future_data[future_data["will_pay_next_month"] == 1][features]
future_data.loc[future_data["will_pay_next_month"] == 1, "predicted_payment"] = reg.predict(X_reg_future)

# 8. Fill zero for rows with no expected payment
future_data["predicted_payment"] = future_data["predicted_payment"].fillna(0)

# 9. Final forecast output
forecast_result = future_data[[
    "supplier",
    "mpan_mprn",
    "month_period",
    "will_pay_next_month",
    "predicted_payment"
]]


forecast_result
# # Optional: sort or export
# print(forecast_result.head())
# # forecast_result.to_csv("predicted_supplier_payments.csv", index=False)


,supplier,mpan_mprn,month_period,will_pay_next_month,predicted_payment
0,EDF,1848605,2024-05,0,0.0
1,EDF,1848605,2024-06,0,0.0
2,EDF,1848605,2024-07,0,0.0
3,EDF,1848605,2024-08,0,0.0
4,EDF,1848605,2024-09,0,0.0
...,...,...,...,...,...
196181,TotalEnergies,2700001843909,2026-03,0,0.0
196182,TotalEnergies,2700001843909,2026-04,0,0.0
196183,TotalEnergies,2700001843909,2026-05,0,0.0
196184,TotalEnergies,2700001843909,2026-06,0,0.0


In [86]:
forecast_result.to_csv("forecast_result.csv", index=False)

from google.colab import files
files.download("forecast_result.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>